F2 test of similarity of correlation coefficients

In [1]:
!pip install xarray

-------- freesurfer-darwin-macOS-dev-20221215-995cb7d --------
Setting up environment for FreeSurfer/FS-FAST (and FSL)
FREESURFER_HOME   /Applications/freesurfer
FSFAST_HOME       /Applications/freesurfer/fsfast
FSF_OUTPUT_FORMAT nii.gz
SUBJECTS_DIR      /Applications/freesurfer/subjects
MNI_DIR           /Applications/freesurfer/mni
FSL_DIR           /Users/cu135/fsl
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install xarray

-------- freesurfer-darwin-macOS-dev-20221215-995cb7d --------
Setting up environment for FreeSurfer/FS-FAST (and FSL)
FREESURFER_HOME   /Applications/freesurfer
FSFAST_HOME       /Applications/freesurfer/fsfast
FSF_OUTPUT_FORMAT nii.gz
SUBJECTS_DIR      /Applications/freesurfer/subjects
MNI_DIR           /Applications/freesurfer/mni
FSL_DIR           /Users/cu135/fsl
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import pymc as pm
import arviz as az

def bayesian_f2_test(r1, n1, r2, n2, num_samples=1000):
    """
    Perform a Bayesian f^2 test to determine if two correlations are more similar than expected by chance.

    Parameters:
    r1 (float): Correlation coefficient of the first sample.
    n1 (int): Sample size of the first sample.
    r2 (float): Correlation coefficient of the second sample.
    n2 (int): Sample size of the second sample.
    num_samples (int): Number of posterior samples to draw.

    Returns:
    az.InferenceData: The posterior inference data.
    """
    
    # Fisher Z-transform
    Z1 = 0.5 * np.log((1 + r1) / (1 - r1))
    Z2 = 0.5 * np.log((1 + r2) / (1 - r2))

    # Calculate the variances of the Fisher Z-transformed correlations
    var_z1 = 1 / (n1 - 3)
    var_z2 = 1 / (n2 - 3)

    # Combined variance of the difference
    var_diff = var_z1 + var_z2
    sigma_diff = np.sqrt(var_diff)
    
    # Observed difference in Z-scores
    Z_diff = Z1 - Z2

    # Bayesian Model
    with pm.Model() as model:
        # Prior for delta (the difference in Z-scores)
        delta = pm.Normal('delta', mu=0, sigma=1)
        
        # Likelihood
        likelihood = pm.Normal('likelihood', mu=delta, sigma=sigma_diff, observed=Z_diff)
        
        # Posterior Inference
        trace = pm.sample(num_samples, return_inferencedata=True)

    return trace


ModuleNotFoundError: No module named 'xarray'

In [ ]:

# Example usage
r1, n1 = 0.6, 100  # Correlation and sample size for the first sample
r2, n2 = 0.65, 120  # Correlation and sample size for the second sample
trace = bayesian_f2_test(r1, n1, r2, n2)

# Plotting posterior distribution of delta
az.plot_posterior(trace, var_names=['delta'])
